In [51]:
#AI-TECHGYM-1-8-Q
#自然言語処理

from janome.tokenizer import Tokenizer

import os
os.chdir("C:/Users/tsuchida/Documents/techgym_セミナー/TortoiseGit_resorce/techgym_ai/Chapter_1/Answer_sheet/AI_Chapter1_saved_files")

f = open('techgym-AI.txt')
s = f.read()
print(s)
print()
print('以下、形態素解析の結果')
print('**************************************')

t = Tokenizer()
tokens = t.tokenize(s)

for token in tokens:
  print(token)

f.close()

月額2万円の「プログラミングライフ」
「テックジム（TechGYM）」は、どんなテクノロジーにも即座に対応できる「自走できるエンジニア」をゴールにした自習形式のプログラマー養成塾です。
どんなレベルからでも始められますので、全くのプログラミング初心者からベテランエンジニアまで、各個人のスキルレベルにあわせて学ぶことができます。
なお、最速でプログラミングスキルの獲得をしたい方にはテックジムのカリキュラムがオススメです。
開催期間にあわせて、好きな時間に好きなだけ、何回でも参加することが可能で、課題を持ち帰ることも可能です。（提携校により開催頻度や開催時間が違います。）
自習を見守るトレーナーは、経験豊富な現役プログラマーです。カリキュラム範囲以外でも、当人の得意分野であれば、技術的な相談相手になってくれますので、「卒業」という概念もありません。会員同士の交流は自由ですので、エンジニア仲間や生涯の仲間を見つける感覚でご参加ください。
まさに、いつでも参加でき、ずっとスキルを磨き続けることのできる「ジム」の要素が「テックジム（TechGYM）」の大きな特徴です。
入会金は3万円（現在は無料）、月額2万円の「プログラミングライフ」をお楽しみください。
もっとも効率的なスキル習得「テックジム方式」とは？
「テックジム方式」とは、基礎知識なしでも、座学なしでプログラミングに専念できるように設計されたプログラミングのカリキュラムメソッドです。
優れたエンジニアの多くは、職業訓練学校や研修スクールで学ぶような学び方をしません。
作りたいものを作っていたら、自然に習得できているのです。そして、優れた指導者（メンターや師匠）との出会いがエンジニアとしてのプロフェッショナル性を高めます。
この自発性とプロフェッショナル性を兼ね備えたのが「テックジム方式」です。
授業を聞いたりテキストを読むことで失う時間を、純粋にプログラミング時間に向けることで、習得効率は飛躍的に向上し、モチベーションも維持できます。
その時点で知らなくていい知識や概念を可能な限り削ぎ落とし、知るべきタイミングが訪れた時にはすでに課題を通じて覚えているようなスキル習得の自動化を図ります。
講座で用いるサンプルソースや課題は、最も効率よく学べるように細部に渡って設計されています。
まるで魔法にかかったようにプログラミンス

し	動詞,自立,*,*,サ変・スル,連用形,する,シ,シ
ませ	助動詞,*,*,*,特殊・マス,未然形,ます,マセ,マセ
ん	助動詞,*,*,*,不変化型,基本形,ん,ン,ン
。	記号,句点,*,*,*,*,。,。,。

	記号,空白,*,*,*,*,
,*,*
作り	動詞,自立,*,*,五段・ラ行,連用形,作る,ツクリ,ツクリ
たい	助動詞,*,*,*,特殊・タイ,基本形,たい,タイ,タイ
もの	名詞,非自立,一般,*,*,*,もの,モノ,モノ
を	助詞,格助詞,一般,*,*,*,を,ヲ,ヲ
作っ	動詞,自立,*,*,五段・ラ行,連用タ接続,作る,ツクッ,ツクッ
て	助詞,接続助詞,*,*,*,*,て,テ,テ
い	動詞,非自立,*,*,一段,連用形,いる,イ,イ
たら	助動詞,*,*,*,特殊・タ,仮定形,た,タラ,タラ
、	記号,読点,*,*,*,*,、,、,、
自然	名詞,形容動詞語幹,*,*,*,*,自然,シゼン,シゼン
に	助詞,副詞化,*,*,*,*,に,ニ,ニ
習得	名詞,サ変接続,*,*,*,*,習得,シュウトク,シュートク
でき	動詞,自立,*,*,一段,連用形,できる,デキ,デキ
て	助詞,接続助詞,*,*,*,*,て,テ,テ
いる	動詞,非自立,*,*,一段,基本形,いる,イル,イル
の	名詞,非自立,一般,*,*,*,の,ノ,ノ
です	助動詞,*,*,*,特殊・デス,基本形,です,デス,デス
。	記号,句点,*,*,*,*,。,。,。
そして	接続詞,*,*,*,*,*,そして,ソシテ,ソシテ
、	記号,読点,*,*,*,*,、,、,、
優れ	動詞,自立,*,*,一段,連用形,優れる,スグレ,スグレ
た	助動詞,*,*,*,特殊・タ,基本形,た,タ,タ
指導	名詞,サ変接続,*,*,*,*,指導,シドウ,シドー
者	名詞,接尾,一般,*,*,*,者,シャ,シャ
（	記号,括弧開,*,*,*,*,（,（,（
メンター	名詞,一般,*,*,*,*,メンター,*,*
や	助詞,並立助詞,*,*,*,*,や,ヤ,ヤ
師匠	名詞,一般,*,*,*,*,師匠,シショウ,シショー
）	記号,括弧閉,*,*,*,*,）,）,）
と	助詞,格助詞,引用,*,*,*,と,ト,ト
の	助詞,連体化,*,*,*,*,の,ノ,ノ
出会い	名詞,一般,*,*,*,*,出

In [50]:
lines = txt.split("\,")
print(type(lines))
print(lines)

<class 'list'>
['月額2万円の「プログラミングライフ」\n「テックジム（TechGYM）」は、どんなテクノロジーにも即座に対応できる「自走できるエンジニア」をゴールにした自習形式のプログラマー養成塾です。\nどんなレベルからでも始められますので、全くのプログラミング初心者からベテランエンジニアまで、各個人のスキルレベルにあわせて学ぶことができます。\nなお、最速でプログラミングスキルの獲得をしたい方にはテックジムのカリキュラムがオススメです。\n開催期間にあわせて、好きな時間に好きなだけ、何回でも参加することが可能で、課題を持ち帰ることも可能です。（提携校により開催頻度や開催時間が違います。）\n自習を見守るトレーナーは、経験豊富な現役プログラマーです。カリキュラム範囲以外でも、当人の得意分野であれば、技術的な相談相手になってくれますので、「卒業」という概念もありません。会員同士の交流は自由ですので、エンジニア仲間や生涯の仲間を見つける感覚でご参加ください。\nまさに、いつでも参加でき、ずっとスキルを磨き続けることのできる「ジム」の要素が「テックジム（TechGYM）」の大きな特徴です。\n入会金は3万円（現在は無料）、月額2万円の「プログラミングライフ」をお楽しみください。\nもっとも効率的なスキル習得「テックジム方式」とは？\n「テックジム方式」とは、基礎知識なしでも、座学なしでプログラミングに専念できるように設計されたプログラミングのカリキュラムメソッドです。\n優れたエンジニアの多くは、職業訓練学校や研修スクールで学ぶような学び方をしません。\n作りたいものを作っていたら、自然に習得できているのです。そして、優れた指導者（メンターや師匠）との出会いがエンジニアとしてのプロフェッショナル性を高めます。\nこの自発性とプロフェッショナル性を兼ね備えたのが「テックジム方式」です。\n授業を聞いたりテキストを読むことで失う時間を、純粋にプログラミング時間に向けることで、習得効率は飛躍的に向上し、モチベーションも維持できます。\nその時点で知らなくていい知識や概念を可能な限り削ぎ落とし、知るべきタイミングが訪れた時にはすでに課題を通じて覚えているようなスキル習得の自動化を図ります。\n講座で用いるサンプルソースや課題は、最も効率よく学べるように細部に渡